<a href="https://colab.research.google.com/github/SherbyRobotics/pyro/blob/colab/examples/notebooks/two_link_robot_with_custom_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template for testing a custom controler on a two-link manipulator

This page is a template for implementing and testing controllers for a two-link planar manipulator. 

## Importing Librairies

In [ ]:
!git clone -b dev-alex https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')

In [ ]:
import pyro
import numpy as np
import matplotlib.pyplot as plt

from IPython import display
!apt install ffmpeg

from pyro.dynamic  import manipulator
from pyro.control  import robotcontrollers

## Defining Dynamic System Model

In [ ]:
robot = manipulator.TwoLinkManipulator()

robot.l1 = 0.5 # length of first rigid link
robot.l2 = 0.3 # length of second rigid link

### Showing the defined robot

In [ ]:
q = [0.1,0.1]     # robot configuration [ joint 1 angle (rad) ,  joint 2 angle (rad)]
robot.show( q )

### Showing the robot natural behavior with no controllers

In [ ]:
robot.x0 = np.array([0.1,0.1,0.0,0.0])  # robot initial states [ joint 1 angle (rad),  joint 2 angle (rad), joint 1 velocity (rad/sec),  joint 1 velocity (rad/sec)]
natural_trajectory = robot.compute_trajectory()

# Animate the simulation
video1 = robot.generate_simulation_html_video()
html1 = display.HTML(video1)
display.display(html1)

## Defining a custom feedback law

In [ ]:
def feedback_law( y , r , t = 0):
  """
  y = [  q0,  q1, dq0 , dq1 ] : Feedback signal  = Robot joint angles and joint velocities
  r = [ qd0, qd1]             : Reference signal = Robot desired joint angles
  """

  u = np.array([2.0,2.0])       # Place-holder to overwrite with your control law

  #u[0] = t
  
  return u



In [ ]:
controller            = robotcontrollers.RobotController( dof = 2 )
controller.c          = feedback_law

Define the desired joint angle

In [ ]:
controller.rbar       = np.array([0.5,0.5])  # Desired robot joint configuration [ joint 1 angle (rad) ,  joint 2 angle (rad)]

## Simulating the robot in closed-loop

In [ ]:
closed_loop_robot = controller + robot

closed_loop_robot.x0 = np.array([0.1,0.1,0,0])  # robot initial states [ joint 1 angle (rad),  joint 2 angle (rad), joint 1 velocity (rad/sec),  joint 1 velocity (rad/sec), PID_internalstate_1 , PID_internalstate_2]
closed_loop_trajectory = closed_loop_robot.compute_trajectory()

# Plot results
closed_loop_robot.plot_trajectory('x')

In [ ]:
closed_loop_robot.plot_trajectory('u')

In [ ]:
# Animate the simulation
video2 = closed_loop_robot.generate_simulation_html_video()
html2 = display.HTML(video2)
display.display(html2)

## Evaluation

In [ ]:
qf = closed_loop_trajectory.x[-1,0:2]
print('Final position=',qf)

ef = qf - controller.rbar
print('Final error=',ef)

score = 100 - np.linalg.norm(ef)
print('Final score=',score)
